In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV

Dataset needs galactic mass, redshift of the galaxy (most of the redshifts are NaN in my set)

In [14]:
data = pd.read_csv("/Datasets/SGcat.csv")

In [16]:
data

,name,sn_type,redshift,mult_sn_flag,galaxy_name,position_flag,morph_type,t_type,inclination,position_angle,redshift_flag,galaxy_app_mag_flag,galaxy_app_mag,log_diameter,class
0,SN 2013K,II P,NaN,NaN,E009-G10,NaN,Sbc,4.0,41.0,171.0,NaN,NaN,12.70,1.32,SUPERNOVA TYPE II
1,SN 2013ch,IIb,NaN,NaN,E007-G06,NaN,Sdm,7.8,47.0,60.0,NaN,NaN,14.70,1.22,SUPERNOVA TYPE II
2,SN 2006ai,II,NaN,NaN,E005-G09,NaN,SBdm,8.0,37.0,112.0,NaN,NaN,15.64,1.02,SUPERNOVA TYPE II
3,SN 2012ah,Ia,NaN,Y,NGC7637,NaN,Sc,5.1,23.0,NaN,NaN,NaN,13.07,1.36,SUPERNOVA TYPE I
4,SN 1992ao,II,NaN,Y,NGC7637,NaN,Sc,5.1,23.0,NaN,NaN,NaN,13.07,1.36,SUPERNOVA TYPE II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,SN 2008gg,Ia,NaN,NaN,NGC 539,NaN,SBc,5.0,32.0,145.0,NaN,NaN,14.29,1.19,SUPERNOVA TYPE I
998,SN 2001et,II,NaN,NaN,M-03-51-09,NaN,SBc,6.5,37.0,145.0,NaN,NaN,14.76,0.98,SUPERNOVA TYPE II
999,SN 1953K,NaN,NaN,NaN,IC4536,G,SBd,7.9,38.0,10.0,NaN,NaN,13.80,1.34,SUPERNOVA
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
usedata = data.groupby(['galaxy_name', 'morph_type', 't_type', 'galaxy_app_mag', 'log_diameter']).size().reset_index(name='Count')

In [50]:
usedata.iloc[:, 1:5]

,morph_type,t_type,galaxy_app_mag,log_diameter
0,SBdm,8.0,15.64,1.02
1,Sdm,7.8,14.70,1.22
2,Sbc,4.0,12.70,1.32
3,Sbc,4.0,15.19,1.28
4,SBbc pec,4.0,13.77,1.30
...,...,...,...,...
536,Sa,1.2,15.93,0.75
537,Sbc,4.3,16.38,0.85
538,Sab,2.1,15.43,0.27
539,Sc,4.8,14.57,1.05


In [20]:
data["morph_type"] = data["morph_type"].str.replace(r":$", "")

C:\Users\shubh\AppData\Local\Temp\ipykernel_29128\18524370.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data["morph_type"] = data["morph_type"].str.replace(r":$", "")


In [21]:
data["sn_type"] = data["sn_type"].str.replace(r":$", "")

C:\Users\shubh\AppData\Local\Temp\ipykernel_29128\2611488093.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data["sn_type"] = data["sn_type"].str.replace(r":$", "")


In [51]:
X = usedata.iloc[:, 1:5]

In [52]:
Y = usedata.iloc[:, 5:6]

In [53]:
random_grid = {'bootstrap': [True, False],
               'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10],
               'n_estimators': [130, 180, 230]}

In [54]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(scoring="neg_mean_squared_error", estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, Y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\shubh\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
300 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\shubh\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\shubh\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 327, in fit
    X, y = self._validate_data(
  File "C:\Users\shubh\anaconda3\lib\site-packages\sklearn\base.py", line 581, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "C:\Users\shubh\anaconda3\lib\

ValueError: could not convert string to float: 'SBdm'